In [2]:
import pandas as pd
import glob

#print(glob.glob("/home/jovyan/*"))
#allFiles = glob.glob("/home/jovyan/work/data/all_0000000000*")
allFiles = glob.glob("data/all_0000000000*")

print(allFiles)
data = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)
data = pd.concat(list_)


['data/all_000000000013', 'data/all_000000000014', 'data/all_000000000012', 'data/all_000000000008', 'data/all_000000000001', 'data/all_000000000006', 'data/all_000000000007', 'data/all_000000000000', 'data/all_000000000009', 'data/all_000000000010', 'data/all_000000000011', 'data/all_000000000005', 'data/all_000000000002', 'data/all_000000000003', 'data/all_000000000004']


In [3]:
def filter_train_type(df=[],
                      train_types=[],
                      sum_types = False,
                      sum_columns=['train_count', 'delay', 'total_delay'],
                      aggs={'pressure': 'max',
                            'max_temperature': 'max',
                            'min_temperature': 'min',
                            'mean_temperature': 'mean',
                            'mean_dewpoint': 'mean',
                            'mean_humidity': 'mean',
                            'mean_winddirection': 'mean',
                            'mean_windspeed': 'mean',
                            'max_windspeedms': 'max',
                            'max_windgust': 'max',
                            'max_precipitation1h': 'max',
                            'max_snowdpeth': 'max',
                            'max_n': 'max',
                            'min_vis': 'min',
                            'min_clhb': 'min',
                            'max_precipitation3h': 'max',
                            'max_precipitation6h': 'max'}):
    """
    Filter traintypes from metadata

    train_types     : list
                      list of following options: [0,1,2,3]

                      train_types = {'K': 0,
                                     'L': 1,
                                     'T': 2,
                                     'M': 3}
    sum_types       : bool
                     if True, sum different train types together (default False)

    returns : np array, np array
              labels metadata, labels
    """

    if len(df) == 0:
        return df

    mask = df.loc[:,'train_type'].isin(train_types)
    filt_df = df[(mask)]

    if sum_types:
        d = {}
        for col in sum_columns:
            d[col] = ['sum']
        for col,method in aggs.items():
            d[col] = [method]
        if 'lat' in df:
            d['lat'] = ['max']
        if 'lon' in df:
            d['lon'] = ['max']

        filt_df = filt_df.groupby(['trainstation', 'time'], as_index=False).agg(d)

        filt_df.columns = filt_df.columns.droplevel(1)
        filt_df.drop_duplicates(['trainstation', 'time'], inplace=True)

    return filt_df

data_filt = filter_train_type(data, ['K', 'L'])

In [11]:
print(data_filt)

                            time trainstation train_type  train_count  \
0        2013-08-04 13:00:00 UTC          HPK          K            1   
2        2012-02-01 17:00:00 UTC          KKN          K            1   
4        2010-12-15 07:00:00 UTC          VAR          K            1   
5        2013-06-01 13:00:00 UTC          ENO          K            1   
6        2011-01-30 20:00:00 UTC          KVU          K            1   
7        2015-07-03 15:00:00 UTC          IKY          K            1   
8        2010-05-19 08:00:00 UTC          KVU          K            1   
9        2016-01-06 23:00:00 UTC          HKP          K            1   
11       2014-01-17 21:00:00 UTC          ÄHT          K            1   
13       2010-07-12 12:00:00 UTC          RKI          K            1   
16       2010-07-29 06:00:00 UTC          TJA          K            1   
18       2010-07-16 10:00:00 UTC          HIR          K            1   
19       2012-02-02 12:00:00 UTC          ROI      

In [3]:
data_filt.sort_values(by=['time', 'trainstation'], inplace=True)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [5]:
data_filt_time = data_filt.loc[(data_filt['time'] >= '2010-01-01') & (data_filt['time'] <= '2010-03-31')]

In [8]:
data_filt_tiny = data_filt.loc[(data_filt['time'] >= '2010-01-01') & (data_filt['time'] <= '2010-01-10')]

In [4]:
data_validation = data_filt.loc[(data_filt['time'] >= '2011-01-01') & (data_filt['time'] <= '2011-01-10')]

In [13]:
print(data_validation)

                            time trainstation train_type  train_count  \
6        2011-01-30 20:00:00 UTC          KVU          K            1   
25       2011-01-26 09:00:00 UTC          UIM          K            1   
57       2011-01-15 22:00:00 UTC          KKN          K            1   
81       2011-01-05 07:00:00 UTC          PTO          K            1   
125      2011-01-04 13:00:00 UTC          LAI          K            1   
178      2011-01-05 22:00:00 UTC          PTO          K            1   
356      2011-01-05 16:00:00 UTC          PTO          K            1   
404      2011-01-16 08:00:00 UTC          LIS          K            1   
539      2011-01-05 08:00:00 UTC          KVU          K            1   
680      2011-01-14 14:00:00 UTC          MUL          K            1   
715      2011-01-14 22:00:00 UTC          HPK          K            1   
852      2011-01-30 19:00:00 UTC          ENO          K            1   
893      2011-01-15 23:00:00 UTC          LAI      

In [26]:
data_filt.to_csv('/home/jovyan/work/data/all_filtered.csv', encoding='utf-8')

In [6]:
data_filt_time.to_csv('/home/jovyan/work/data/2010q1_filtered.csv', encoding='utf-8')

In [10]:
data_filt_tiny.to_csv('/home/jovyan/work/data/tiny_filtered.csv', encoding='utf-8')

In [5]:
data_validation.to_csv('data/validation.csv', encoding='utf-8')